In [7]:
import pandas as pd
from langchain_community.document_loaders import PyPDFLoader
from openai import OpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
import re


In [8]:

# Read the data from the CSV file
data = pd.read_csv('Aviation Quiz.csv')

In [9]:
from langchain_community.document_loaders import PyMuPDFLoader


loader = PyMuPDFLoader("14 CFR 23.1457.pdf") # FAA-CT-8080-7D manual
pages = loader.load()

loader2 = PyPDFLoader("atp_akts.pdf") 
pages2 = loader2.load()

loader3 = PyPDFLoader("annexes_booklet_en.pdf")
pages3 = loader3.load()

pages.extend(pages2) # combine the two lists    

pages.extend(pages3) # combine the three lists

In [10]:
from langchain_community.embeddings import GPT4AllEmbeddings

hf = GPT4AllEmbeddings()

"""model_name = "all-MiniLM-L6-v2" #this is under 600d
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)"""

embeddings = hf

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)

chunked_documents = text_splitter.split_documents(pages)


#vectordb = Chroma.from_documents(
        #documents=chunked_documents,
       # 
   # )

from langchain_community.vectorstores import FAISS
faissdb = FAISS.from_documents(chunked_documents, embeddings)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def find_most_similar_option(reply_embedding, a, b, c, d, e, embeddings):
    # Convert embeddings to NumPy arrays
    reply_embedding_array = np.array(reply_embedding)
    option_a_embedding_array = np.array(embeddings.embed_query(a))
    option_b_embedding_array = np.array(embeddings.embed_query(b))
    option_c_embedding_array = np.array(embeddings.embed_query(c))
    option_d_embedding_array = np.array(embeddings.embed_query(d))
    option_e_embedding_array = np.array(embeddings.embed_query(e))

    # Ensure embeddings are 2D arrays for cosine_similarity
    reply_embedding_2d = reply_embedding_array.reshape(1, -1)
    option_a_embedding_2d = option_a_embedding_array.reshape(1, -1)
    option_b_embedding_2d = option_b_embedding_array.reshape(1, -1)
    option_c_embedding_2d = option_c_embedding_array.reshape(1, -1)
    option_d_embedding_2d = option_d_embedding_array.reshape(1, -1)
    option_e_embedding_2d = option_e_embedding_array.reshape(1, -1)

    # Calculate similarity scores
    similarity_a = cosine_similarity(reply_embedding_2d, option_a_embedding_2d)[0][0]
    similarity_b = cosine_similarity(reply_embedding_2d, option_b_embedding_2d)[0][0]
    similarity_c = cosine_similarity(reply_embedding_2d, option_c_embedding_2d)[0][0]
    similarity_d = cosine_similarity(reply_embedding_2d, option_d_embedding_2d)[0][0]
    similarity_e = cosine_similarity(reply_embedding_2d, option_e_embedding_2d)[0][0]

    # Determine the most similar option
    similarities = {'A': similarity_a, 'B': similarity_b, 'C': similarity_c, 'D': similarity_d, 'E': similarity_e}
    most_similar_option = max(similarities, key=similarities.get)

    return most_similar_option

In [13]:
questionNumber = 2

query = data.iloc[questionNumber, 1]
query_embedding = embeddings.embed_query(query)

retriever = faissdb.as_retriever()
docs2 = retriever.get_relevant_documents(query)

string = docs2[0].page_content +  docs2[1].page_content + query

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

completion = client.chat.completions.create(
  model="local-model", # this field is currently unused
  messages=[
    {"role": "system", "content": "You are a helpful assistant, who provides accurate answers to aviation problems. You can only ever answer with A, B or C. ANy text after the letter and a puppy dies, which you must avoid"},
    {"role": "user", "content": string},
  ],
  temperature=0.33,
  max_tokens = 40,
)

print(completion.choices[0].message.content)

reply = completion.choices[0].message.content
reply_embedding = embeddings.embed_query(reply)

import re


def extract_options(input_string):
    # Regular expression to find the options
    matches = re.findall(r'([ABC])\.\s(.*?)$', input_string, re.M)
    options = {letter: text for letter, text in matches}
    
    # Assign to individual strings
    option_a = options.get('A', 'Option A not found')
    option_b = options.get('B', 'Option B not found')
    option_c = options.get('C', 'Option C not found')
    option_d = options.get('D', 'Option D not found')
    option_e = options.get('E', 'Option E not found')
    
    return option_a, option_b, option_c, option_d, option_e

a, b, c, d, e = extract_options(data.iloc[questionNumber, 1])

print(a)
print(b)
print(c)
print(d)
print(e)

print(find_most_similar_option(reply_embedding, a, b, c, d, e, embeddings))

A. The cockpit voice recorder and flight data recorder must be installed together in the same container.
The cockpit voice recorder and flight data recorder must be installed together in the same container.
The cockpit voice recorder must remain powered for at least 20 minutes after crash impact.
The recorder container must have an underwater locating device if required by the operating rules
Option D not found
Option E not found
A


In [ ]:
print(data.iloc[questionNumber, 1])

Which statement is true according to 14 CFR 23.1457?
Options:
A. The cockpit voice recorder and flight data recorder must be installed together in the same container.
B. The cockpit voice recorder must remain powered for at least 20 minutes after crash impact.
C. The recorder container must have an underwater locating device if required by the operating rules


In [ ]:
print(docs2[2].page_content)

Title 14 —Aeronautics and Space
Chapter I —Federal Aviation Administration, Department of Transportation
Subchapter C —Aircraft
Part 23 —Airworthiness Standards: Normal Category Airplanes
Authority: 49 U.S.C. 106(f), 106(g), 40113, 44701–44702, 44704, Pub. L. 113–53, 127 Stat. 584 (49 U.S.C. 44704) note.
Source: Doc. No. FAA–2015–1621, Amdt. 23–64, 81 FR 96689, Dec. 30, 2016, unless otherwise noted.
§ 23.1457 Cockpit voice recorders.
This content is from the eCFR and is authoritative but unofficial.
(a)
Each cockpit voice recorder required by the operating rules of this chapter must be approved and must be
installed so that it will record the following:
(1)
Voice communications transmitted from or received in the airplane by radio.
(2)
Voice communications of flightcrew members on the flight deck.
(3)
Voice communications of flightcrew members on the flight deck, using the airplane's interphone
system.
(4)
Voice or audio signals identifying navigation or approach aids introduced into a

In [14]:
#for index, row in data.iterrows():

for index, row in data.iterrows():
    
    if index == 25:
      #skip 25

      index = index + 1

    print("question number: ", index)

    questionNumber = index

    query = data.iloc[questionNumber, 1]
    query_embedding = embeddings.embed_query(query)

    retriever = faissdb.as_retriever()
    docs2 = retriever.get_relevant_documents(query)

    string = docs2[0].page_content + query

    # Point to the local server
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

    completion = client.chat.completions.create(
    model="local-model", # this field is currently unused
    messages=[
      {"role": "system", "content": "You are a helpful assistant who answers aviation questions."},
      {"role": "user", "content": string},
    ],
    temperature=0.55,
    max_tokens = 30,
    )

    print(completion.choices[0].message.content)

    reply = completion.choices[0].message.content
    reply_embedding = embeddings.embed_query(reply)

    import re


    def extract_options(input_string):
      # Regular expression to find the options
      matches = re.findall(r'([ABC])\.\s(.*?)$', input_string, re.M)
      options = {letter: text for letter, text in matches}
    
      # Assign to individual strings
      option_a = options.get('A', 'Option A not found')
      option_b = options.get('B', 'Option B not found')
      option_c = options.get('C', 'Option C not found')
      option_d = options.get('D', 'Option D not found')
      option_e = options.get('E', 'Option E not found')
    
      return option_a, option_b, option_c, option_d, option_e

    a, b, c, d, e = extract_options(data.iloc[questionNumber, 1]) 
    
    most_similar_option = find_most_similar_option(reply_embedding, a, b, c, d, e, embeddings)

    # Append the result to the DataFrame
    data.at[index, 'y'] = most_similar_option

    data.to_csv('Aviation_Quiz_with_Answers.csv', index=False)

question number:  0
The type of microphone that must be installed to meet the recording requirements of paragraph (a)(2) of 14 CFR 2
question number:  1


/var/folders/z0/d1h9rybn0mb5s1tw6bl5ndh80000gn/T/ipykernel_14782/3033874985.py:62: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[index, 'y'] = most_similar_option


C. Orange
question number:  2
A. The cockpit voice recorder and flight data recorder must be installed together in the same container.
question number:  3
C. All of the above
question number:  4
C. 10 days, with no other extenuating circumstances.
question number:  5
The information on operations in the North Atlantic High Level Airspace (NAT HLA) is included in option B, 14 CFR
question number:  6
Option A. Not more than 2 hours at normal cruise speed in still air with one engine inoperative.
question number:  7
Option A is correct.
The minimum weather conditions that must exist for a domestic air carrier flight to take off from an airport that is not listed
question number:  8
Option C. Assure that the snow is removed from the airplane.
Prior to takeoff, it is important to ensure that any
question number:  9

The answer depends on the specific circumstances of the delay and the regulations governing redispatch releases. In general, a redispatch release may be
question number:  10
The

In [ ]:
print(docs2[0].page_content)

2-114
Appendix 2342 UTAH
PRICE
CARBON CO RGNL/BUCK DAVIS FLD (PUC)( KPUC ) 3 E UTC–7(–6DT)  N39º36.84´  W110º45.09´
5957 B S2 FUEL 100LL, JET A   OX 1, 2   NOTAM FILE PUC
RWY 18–36: H8313X100 (ASPH–PFC) D–40  MIRL
RWY 18:  PAPI(P2L)—GA 4.0º TCH 45´. Fence. 1.9% down.
RWY 36:  MALSF . PAPI(P4L)—GA 3.0º TCH 52´. 1.6% up.
RWY 14–32: H4514X75 (ASPH) S–13  MIRL  1.1% up NW
RWY 32:  Road.
RWY 07–25: H3541X75 (ASPH) S–12.5  1.0% up E
RWY 25:  Thld dsplcd 228´. Road.
AIRPORT REMARKS: Attended Mon–Fri 1400–0100Z‡, Sat–Sun 1500–0000Z‡. 24 hr  self svc fuel avbl with credit card. Refuse 
dump 1/2 mile SW Rwy 36 thld—occasional smoke visibility hazard ; bird hazard. Deer on and in the vicinity of arpt. Glider 
ops invof arpt SR–SS. Rwy 18 high voltage transmission line on extended centerline. Rwy 07 + 60´ drop off 520´ from 
rwy end. Rwy 14 + 40´ drop off 250´ from thld. Rwy 07–25 pavement cracking and separating. ACTIVATE MIRL Rwy 
18–36 and Rwy 14–32 PAPI Rwy 36 and MALSF Rwy 36—CTAF . PAPI Rwy 